<a href="https://colab.research.google.com/github/karthikravi123/pytorch_practice/blob/main/Multiclass_classification_pytorch_custom_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [39]:
iris = load_iris()
df = pd.DataFrame(iris.data,columns=iris.feature_names)

In [40]:
df.head(2)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2


In [41]:
df['target']  = iris.target

In [42]:
df.head(2)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0


In [43]:
train_df, test_df = train_test_split(df,test_size=0.2,random_state=42,stratify=df['target'])

**What stratify does**

stratify ensures that the train and test sets have the same proportion of classes (or categories) as the original dataset.

It’s mostly used when your target variable is categorical (classification tasks).

Example:

df['target'].value_counts()
 0    80
 1    20


Without stratify: after splitting, the proportion of 0s and 1s might be uneven in train/test.

With stratify=df['target']:

Both train and test sets will roughly maintain the 80:20 ratio

In [44]:
X_train,y_train = train_df.drop(columns= ['target']),train_df['target']

X_test,y_test = test_df.drop(columns= ['target']),test_df['target']

In [45]:
scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [46]:
X_train_tensor = torch.tensor(X_train,dtype=torch.float32)

X_test_tensor = torch.tensor(X_test,dtype=torch.float32)

y_test_tensor = torch.tensor(y_test.values,dtype=torch.long)

y_train_tensor = torch.tensor(y_train.values,dtype=torch.long)

In [47]:
class IrisClassifier(nn.Module):
  def __init__(self,input_dim,hidden_dim,output_dim):
    super(IrisClassifier,self).__init__()

    self.network = nn.Sequential(
        nn.Linear(input_dim,hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim,hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim,output_dim)
    )
  def forward(self,x):
      return self.network(x)

In [48]:
X_train.shape

##total number of values -120 ,total number of features = 4

(120, 4)

In [49]:
input_dim = X_train.shape[1]
input_dim

4

In [50]:
hidden_dim =16
output_dim = 3

In [51]:
model = IrisClassifier(input_dim,hidden_dim,output_dim)

In [52]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)

In [53]:
#train the model

epochs = 500

for epoch in range(epochs):
  model.train()

  optimizer.zero_grad()
  prediction = model(X_train_tensor)
  loss = criterion(prediction,y_train_tensor)
  loss.backward()
  optimizer.step()

  if(epoch+1)%50 == 0 :
    print(f"Epochs {epoch+1}/{epoch} , {loss.item():.4f}")

Epochs 50/49 , 0.0868
Epochs 100/99 , 0.0329
Epochs 150/149 , 0.0261
Epochs 200/199 , 0.0122
Epochs 250/249 , 0.0037
Epochs 300/299 , 0.0016
Epochs 350/349 , 0.0009
Epochs 400/399 , 0.0006
Epochs 450/449 , 0.0004
Epochs 500/499 , 0.0003


In [55]:
model.eval()
with torch.no_grad():
  y_pred = model(X_test_tensor)

  y_pred_labels = torch.argmax(y_pred,dim=1)

  accuracy = (y_pred_labels == y_test_tensor).sum().item()/ y_test_tensor.size(0)
  print(f"test accuracy is :{accuracy:.4f}")


test accuracy is :0.9667


In [56]:
#inference on model on single row

def predict_iris(sepal_length,sepal_width,petal_length,petal_width):
  input_data = np.array([[sepal_length,sepal_width,petal_length,petal_width]])
  input_data = scaler.transform(input_data)
  input_tensor= torch.tensor(input_data,dtype=torch.float32)
  model.eval()

  with torch.no_grad():
    prediction = model(input_tensor)

    prediction_class = torch.argmax(prediction,dim=1).item()

  return iris.target_names[prediction_class]


In [57]:
print(predict_iris(5.1,3.5,1.4,0.2))

setosa


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
